In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, MetaData

In [2]:
# Create the engine
engine = create_engine("sqlite:///hawaii.sqlite")

# Metadata to get the table columns
metadata = MetaData()

# reflect an existing database into a new model
Base = automap_base()

# reflect the entire database
Base.prepare(engine, reflect=True)

# Creating and Saving Objects to reference each table in the database
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
session.query(Measurement.tobs).filter(extract ('month', Measurement.date) == 6)

In [5]:
#  3. Convert the June temperatures to a list.
june_temp = [temp for temp in session.query(Measurement.tobs).filter(extract ('month', Measurement.date) == 6)]

In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_temperatures = pd.DataFrame(june_temp,columns=['June Temps'])

In [8]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
jun = june_temperatures.describe()
jun

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
session.query(Measurement.tobs).filter(extract ('month', Measurement.date) == 12)

In [9]:
# 7. Convert the December temperatures to a list.
december_temp = [temp for temp in session.query(Measurement.tobs).filter(extract ('month', Measurement.date) == 12)]

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
december_temperatures = pd.DataFrame(december_temp,columns=['December Temps'])

In [11]:
# 9. Calculate and print out the summary statistics for the December temperature DataFrame.
dec = december_temperatures.describe()
dec

,December Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [12]:
# Combined columns
a = pd.merge(jun, dec, on=jun.index)
a.columns = ['Stats', 'June Temp', 'December Temp']
a.set_index(a['Stats'], inplace=True)
print(a.to_string(index=False))


 Stats    June Temp  December Temp
 count  1700.000000    1517.000000
  mean    74.944118      71.041529
   std     3.257417       3.745920
   min    64.000000      56.000000
   25%    73.000000      69.000000
   50%    75.000000      71.000000
   75%    77.000000      74.000000
   max    85.000000      83.000000


### Summary Statistics for June and December Precipitation

In [13]:
# Querying the database engine for the precipitation of both months
# Creating a list for both months

june_prcp = [june for june in session.query(Measurement.prcp).filter(extract ('month', Measurement.date) == 6).all()]

dec_prcp = [dec for dec in session.query(Measurement.prcp).filter(extract ('month', Measurement.date) == 12).all()]

In [14]:
# Creating the Dataframes
june_prcp_df = pd.DataFrame(june_prcp)
dec_prcp_df = pd.DataFrame(dec_prcp)

# Getting the statistics and combining the statistics column.
# Then setting the index as the 'Stats' column
prcp_stats_combined = june_prcp_df.describe().merge(dec_prcp_df.describe(), on=june_prcp_df.describe().index)
prcp_stats_combined.columns = ['Stats', 'June Prcp', 'December Prcp']
prcp_stats_combined.set_index(prcp_stats_combined['Stats'], inplace=True)

In [15]:
# Printing the combined precipitation stats dataframe
print(prcp_stats_combined.to_string(index=False))


 Stats    June Prcp  December Prcp
 count  1574.000000    1405.000000
  mean     0.136360       0.216819
   std     0.335731       0.541399
   min     0.000000       0.000000
   25%     0.000000       0.000000
   50%     0.020000       0.030000
   75%     0.120000       0.150000
   max     4.430000       6.420000
